# Alur Kerja Human-in-the-Loop dengan Microsoft Agent Framework

## 🎯 Tujuan Pembelajaran

Dalam notebook ini, Anda akan belajar cara menerapkan alur kerja **human-in-the-loop** menggunakan `RequestInfoExecutor` dari Microsoft Agent Framework. Pola yang kuat ini memungkinkan Anda untuk menghentikan alur kerja AI guna mengumpulkan masukan manusia, menjadikan agen Anda interaktif dan memberikan kontrol kepada manusia atas keputusan penting.

## 🔄 Apa itu Human-in-the-Loop?

**Human-in-the-loop (HITL)** adalah pola desain di mana agen AI menghentikan eksekusi untuk meminta masukan manusia sebelum melanjutkan. Ini sangat penting untuk:

- ✅ **Keputusan penting** - Mendapatkan persetujuan manusia sebelum mengambil tindakan penting
- ✅ **Situasi ambigu** - Membiarkan manusia memberikan klarifikasi saat AI merasa ragu
- ✅ **Preferensi pengguna** - Meminta pengguna memilih di antara beberapa opsi
- ✅ **Kepatuhan & keamanan** - Memastikan pengawasan manusia untuk operasi yang diatur
- ✅ **Pengalaman interaktif** - Membangun agen percakapan yang merespons masukan pengguna

## 🏗️ Cara Kerjanya di Microsoft Agent Framework

Framework ini menyediakan tiga komponen utama untuk HITL:

1. **`RequestInfoExecutor`** - Executor khusus yang menghentikan alur kerja dan memancarkan `RequestInfoEvent`
2. **`RequestInfoMessage`** - Kelas dasar untuk payload permintaan yang diketik dan dikirim ke manusia
3. **`RequestResponse`** - Menghubungkan respons manusia dengan permintaan asli menggunakan `request_id`

**Pola Alur Kerja:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Contoh Kita: Pemesanan Hotel dengan Konfirmasi Pengguna

Kita akan membangun alur kerja bersyarat dengan menambahkan konfirmasi manusia **sebelum** menyarankan destinasi alternatif:

1. Pengguna meminta destinasi (misalnya, "Paris")
2. `availability_agent` memeriksa apakah kamar tersedia
3. **Jika tidak ada kamar** → `confirmation_agent` bertanya "Apakah Anda ingin melihat alternatif?"
4. Alur kerja **berhenti** menggunakan `RequestInfoExecutor`
5. **Manusia merespons** "ya" atau "tidak" melalui input konsol
6. `decision_manager` mengarahkan berdasarkan respons:
   - **Ya** → Menampilkan destinasi alternatif
   - **Tidak** → Membatalkan permintaan pemesanan
7. Menampilkan hasil akhir

Ini menunjukkan bagaimana memberikan kontrol kepada pengguna atas saran agen!

---

Mari kita mulai! 🚀


## Langkah 1: Impor Library yang Dibutuhkan

Kita mengimpor komponen standar Agent Framework ditambah **kelas khusus human-in-the-loop**:
- `RequestInfoExecutor` - Executor yang menghentikan alur kerja untuk input manusia
- `RequestInfoEvent` - Event yang dipancarkan saat input manusia diminta
- `RequestInfoMessage` - Kelas dasar untuk payload permintaan yang terketik
- `RequestResponse` - Menghubungkan respons manusia dengan permintaan
- `WorkflowOutputEvent` - Event untuk mendeteksi output alur kerja


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Langkah 2: Definisikan Model Pydantic untuk Output Terstruktur

Model-model ini mendefinisikan **skema** yang akan dikembalikan oleh agen. Kami mempertahankan semua model dari alur kerja bersyarat dan menambahkan:

**Baru untuk Human-in-the-Loop:**
- `HumanFeedbackRequest` - Subkelas dari `RequestInfoMessage` yang mendefinisikan payload permintaan yang dikirimkan kepada manusia
  - Berisi `prompt` (pertanyaan yang akan diajukan) dan `destination` (konteks tentang kota yang tidak tersedia)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Langkah 3: Buat Alat Pemesanan Hotel

Alat yang sama dari alur kerja bersyarat - memeriksa apakah kamar tersedia di tujuan.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Langkah 4: Tentukan Fungsi Kondisi untuk Routing

Kita membutuhkan **empat fungsi kondisi** untuk alur kerja human-in-the-loop kita:

**Dari alur kerja bersyarat:**
1. `has_availability_condition` - Mengarahkan ketika hotel TERSEDIA
2. `no_availability_condition` - Mengarahkan ketika hotel TIDAK TERSEDIA

**Baru untuk human-in-the-loop:**
3. `user_wants_alternatives_condition` - Mengarahkan ketika pengguna mengatakan "ya" untuk alternatif
4. `user_declines_alternatives_condition` - Mengarahkan ketika pengguna mengatakan "tidak" untuk alternatif


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Langkah 5: Membuat Decision Manager Executor

Ini adalah **inti dari pola human-in-the-loop**! `DecisionManager` adalah `Executor` khusus yang:

1. **Menerima umpan balik manusia** melalui objek `RequestResponse`
2. **Memproses keputusan pengguna** (ya/tidak)
3. **Mengatur alur kerja** dengan mengirimkan pesan ke agen yang sesuai

Fitur utama:
- Menggunakan dekorator `@handler` untuk mengekspos metode sebagai langkah alur kerja
- Menerima `RequestResponse[HumanFeedbackRequest, str]` yang berisi permintaan asli dan jawaban pengguna
- Menghasilkan pesan sederhana "ya" atau "tidak" yang memicu fungsi kondisi kita


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Langkah 6: Membuat Eksekutor Tampilan Kustom

Eksekutor tampilan yang sama dari alur kerja bersyarat - menghasilkan hasil akhir sebagai output alur kerja.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Langkah 7: Muat Variabel Lingkungan

Konfigurasikan klien LLM (GitHub Models, Azure OpenAI, atau OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Langkah 8: Membuat Agen AI dan Eksekutor

Kita membuat **enam komponen alur kerja**:

**Agen (dibungkus dalam AgentExecutor):**
1. **availability_agent** - Memeriksa ketersediaan hotel menggunakan alat
2. **confirmation_agent** - 🆕 Menyiapkan permintaan konfirmasi dari manusia
3. **alternative_agent** - Menyarankan kota alternatif (ketika pengguna mengatakan ya)
4. **booking_agent** - Mendorong pemesanan (ketika kamar tersedia)
5. **cancellation_agent** - 🆕 Menangani pesan pembatalan (ketika pengguna mengatakan tidak)

**Eksekutor Khusus:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` yang menghentikan alur kerja untuk input manusia
7. **decision_manager** - 🆕 Eksekutor khusus yang mengarahkan berdasarkan respons manusia (sudah didefinisikan di atas)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Langkah 9: Bangun Alur Kerja dengan Human-in-the-Loop

Sekarang kita membangun grafik alur kerja dengan **pengarahan bersyarat** termasuk jalur human-in-the-loop:

**Struktur Alur Kerja:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Tepi Utama:**
- `availability_agent → confirmation_agent` (ketika tidak ada kamar)
- `confirmation_agent → prepare_human_request` (ubah tipe)
- `prepare_human_request → request_info_executor` (jeda untuk manusia)
- `request_info_executor → decision_manager` (selalu - memberikan RequestResponse)
- `decision_manager → alternative_agent` (ketika pengguna mengatakan "ya")
- `decision_manager → cancellation_agent` (ketika pengguna mengatakan "tidak")
- `availability_agent → booking_agent` (ketika kamar tersedia)
- Semua jalur berakhir di `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Langkah 10: Jalankan Kasus Uji 1 - Kota TANPA Ketersediaan (Paris dengan Konfirmasi Manual)

Tes ini menunjukkan **siklus penuh dengan campur tangan manusia**:

1. Meminta hotel di Paris
2. availability_agent memeriksa → Tidak ada kamar
3. confirmation_agent membuat pertanyaan untuk manusia
4. request_info_executor **menghentikan alur kerja sementara** dan mengeluarkan `RequestInfoEvent`
5. **Aplikasi mendeteksi event dan meminta pengguna melalui konsol**
6. Pengguna mengetik "ya" atau "tidak"
7. Aplikasi mengirimkan respons melalui `send_responses_streaming()`
8. decision_manager mengarahkan berdasarkan respons
9. Hasil akhir ditampilkan

**Pola Utama:**
- Gunakan `workflow.run_stream()` untuk iterasi pertama
- Gunakan `workflow.send_responses_streaming(pending_responses)` untuk iterasi berikutnya
- Dengarkan `RequestInfoEvent` untuk mendeteksi saat input manusia diperlukan
- Dengarkan `WorkflowOutputEvent` untuk menangkap hasil akhir


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Langkah 11: Jalankan Kasus Uji 2 - Kota DENGAN Ketersediaan (Stockholm - Tidak Perlu Input Manusia)

Tes ini menunjukkan **jalur langsung** ketika kamar tersedia:

1. Meminta hotel di Stockholm
2. availability_agent memeriksa → Kamar tersedia ✅
3. booking_agent menyarankan pemesanan
4. display_result menunjukkan konfirmasi
5. **Tidak diperlukan input manusia!**

Alur kerja sepenuhnya melewati jalur human-in-the-loop ketika kamar tersedia.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Poin Penting dan Praktik Terbaik Human-in-the-Loop

### ✅ Apa yang Telah Anda Pelajari:

#### 1. **Pola RequestInfoExecutor**
Pola human-in-the-loop dalam Microsoft Agent Framework menggunakan tiga komponen utama:
- `RequestInfoExecutor` - Menghentikan alur kerja dan memancarkan event
- `RequestInfoMessage` - Kelas dasar untuk payload permintaan yang diketik (subclass ini!)
- `RequestResponse` - Menghubungkan respons manusia dengan permintaan asli

**Pemahaman Penting:**
- `RequestInfoExecutor` TIDAK mengumpulkan input sendiri - hanya menghentikan alur kerja
- Kode aplikasi Anda harus mendengarkan `RequestInfoEvent` dan mengumpulkan input
- Anda harus memanggil `send_responses_streaming()` dengan dict yang memetakan `request_id` ke jawaban pengguna

#### 2. **Pola Eksekusi Streaming**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Arsitektur Berbasis Event**
Dengarkan event tertentu untuk mengontrol alur kerja:
- `RequestInfoEvent` - Input manusia diperlukan (alur kerja dihentikan)
- `WorkflowOutputEvent` - Hasil akhir tersedia (alur kerja selesai)
- `WorkflowStatusEvent` - Perubahan status (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, dll.)

#### 4. **Eksekutor Kustom dengan @handler**
`DecisionManager` menunjukkan cara membuat eksekutor yang:
- Menggunakan dekorator `@handler` untuk mengekspos metode sebagai langkah alur kerja
- Menerima pesan yang diketik (misalnya, `RequestResponse[HumanFeedbackRequest, str]`)
- Mengarahkan alur kerja dengan mengirim pesan ke eksekutor lain
- Mengakses konteks melalui `WorkflowContext`

#### 5. **Pengarahan Bersyarat dengan Keputusan Manusia**
Anda dapat membuat fungsi kondisi yang mengevaluasi respons manusia:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Aplikasi Dunia Nyata:

1. **Alur Kerja Persetujuan**
   - Mendapatkan persetujuan manajer sebelum memproses laporan pengeluaran
   - Memerlukan tinjauan manusia sebelum mengirim email otomatis
   - Mengonfirmasi transaksi bernilai tinggi sebelum eksekusi

2. **Moderasi Konten**
   - Menandai konten yang meragukan untuk tinjauan manusia
   - Meminta moderator membuat keputusan akhir pada kasus batas
   - Meningkatkan ke manusia saat kepercayaan AI rendah

3. **Layanan Pelanggan**
   - Membiarkan AI menangani pertanyaan rutin secara otomatis
   - Meningkatkan masalah kompleks ke agen manusia
   - Menanyakan pelanggan apakah mereka ingin berbicara dengan manusia

4. **Pemrosesan Data**
   - Meminta manusia menyelesaikan entri data yang ambigu
   - Mengonfirmasi interpretasi AI terhadap dokumen yang tidak jelas
   - Membiarkan pengguna memilih antara beberapa interpretasi yang valid

5. **Sistem Kritis Keselamatan**
   - Memerlukan konfirmasi manusia sebelum tindakan yang tidak dapat dibatalkan
   - Mendapatkan persetujuan sebelum mengakses data sensitif
   - Mengonfirmasi keputusan dalam industri yang diatur (kesehatan, keuangan)

6. **Agen Interaktif**
   - Membangun bot percakapan yang mengajukan pertanyaan lanjutan
   - Membuat wizard yang membimbing pengguna melalui proses yang kompleks
   - Merancang agen yang berkolaborasi dengan manusia langkah demi langkah

### 🔄 Perbandingan: Dengan vs Tanpa Human-in-the-Loop

| Fitur | Alur Kerja Bersyarat | Alur Kerja Human-in-the-Loop |
|-------|-----------------------|-----------------------------|
| **Eksekusi** | Satu `workflow.run()` | Loop dengan `run_stream()` + `send_responses_streaming()` |
| **Input Pengguna** | Tidak ada (otomatis sepenuhnya) | Prompt interaktif melalui `input()` atau UI |
| **Komponen** | Agen + Eksekutor | + RequestInfoExecutor + DecisionManager |
| **Event** | Hanya AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, dll. |
| **Penghentian** | Tidak ada penghentian | Alur kerja berhenti di RequestInfoExecutor |
| **Kontrol Manusia** | Tidak ada kontrol manusia | Manusia membuat keputusan penting |
| **Kasus Penggunaan** | Otomasi | Kolaborasi & pengawasan |

### 🚀 Pola Lanjutan:

#### Beberapa Titik Keputusan Manusia
Anda dapat memiliki beberapa node `RequestInfoExecutor` dalam alur kerja yang sama:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Penanganan Timeout
Implementasikan timeout untuk respons manusia:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrasi UI yang Kaya
Alih-alih `input()`, integrasikan dengan UI web, Slack, Teams, dll.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Human-in-the-Loop Bersyarat
Hanya meminta input manusia dalam situasi tertentu:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Praktik Terbaik:

1. **Selalu Subclass RequestInfoMessage**
   - Memberikan keamanan tipe dan validasi
   - Memungkinkan konteks yang kaya untuk rendering UI
   - Memperjelas maksud dari setiap jenis permintaan

2. **Gunakan Prompt yang Deskriptif**
   - Sertakan konteks tentang apa yang Anda tanyakan
   - Jelaskan konsekuensi dari setiap pilihan
   - Buat pertanyaan sederhana dan jelas

3. **Tangani Input yang Tidak Terduga**
   - Validasi respons pengguna
   - Berikan default untuk input yang tidak valid
   - Berikan pesan kesalahan yang jelas

4. **Lacak ID Permintaan**
   - Gunakan korelasi antara request_id dan respons
   - Jangan mencoba mengelola status secara manual

5. **Desain untuk Non-Blocking**
   - Jangan memblokir thread yang menunggu input
   - Gunakan pola async secara keseluruhan
   - Dukung instance alur kerja yang bersamaan

### 📚 Konsep Terkait:

- **Agent Middleware** - Mencegat panggilan agen (notebook sebelumnya)
- **Manajemen Status Alur Kerja** - Menyimpan status alur kerja antar run
- **Kolaborasi Multi-Agen** - Menggabungkan human-in-the-loop dengan tim agen
- **Arsitektur Berbasis Event** - Membangun sistem reaktif dengan event

---

### 🎓 Selamat!

Anda telah menguasai alur kerja human-in-the-loop dengan Microsoft Agent Framework! Anda sekarang tahu cara:
- ✅ Menghentikan alur kerja untuk mengumpulkan input manusia
- ✅ Menggunakan RequestInfoExecutor dan RequestInfoMessage
- ✅ Menangani eksekusi streaming dengan event
- ✅ Membuat eksekutor kustom dengan @handler
- ✅ Mengarahkan alur kerja berdasarkan keputusan manusia
- ✅ Membangun agen AI interaktif yang berkolaborasi dengan manusia

**Ini adalah pola penting untuk membangun sistem AI yang dapat dipercaya dan dapat dikontrol!** 🚀



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang penting, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau interpretasi yang keliru yang timbul dari penggunaan terjemahan ini.
